In [16]:
import datetime
import gc
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import warnings

from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import feather
from utils import reduce_mem_usage,rmse
from FE import train_test,historical_transactions,new_merchant_transactions,additional_features

In [12]:
test_df = feather.read_dataframe("./stack/test_stack.feather")

In [13]:
test = pd.read_csv("./input/test.csv")
final_test = pd.DataFrame(index=test.index)
final_test["card_id"] = test.card_id
final_test["target"] = 0

In [8]:
final_test["target"] = test_df["lgb"]*0.4+test_df["lgb2"]*0.4+test_df["cat"]*0.1+test_df["xgb"]*0.1

In [15]:
final_test.to_csv("nantoka2.csv",index=False)

In [14]:
final_test["target"] =test_df["lgb2"]*1

In [17]:
merchant = feather.read_dataframe("./input/hist_merchant.feather")
merchant.set_index("merchant_id",inplace=True)
merchant1 = feather.read_dataframe("./input/new_merchant.feather")
merchant1.set_index("merchant_id",inplace=True)

df = pd.merge(merchant1, merchant, on='merchant_id', how='outer')

his = df.columns[78:]
new = df.columns[:78]
his = list(his)
new = list(new)
his.sort()
new.sort()
new.remove("hist_subsector_id_nunique")
his.append("hist_subsector_id_nunique")
his.remove("hist_purchase_date_max")
his.remove("hist_purchase_date_min")
new.remove("new_purchase_date_max")
new.remove("new_purchase_date_min")

df = df.fillna(0)
df[his] = df[his].values+df[new].values
hismer = df[his].rename(columns=lambda s: "mer+his" + s)
df[hismer].reset_index().to_feather("./input/newmer+hist_merchant.feather")

ValueError: Must pass DataFrame with boolean values only

In [20]:
hismer

,mer+hishist_Black_Friday_2017_mean,mer+hishist_Children_day_2017_mean,mer+hishist_Christmas_Day_2017_mean,mer+hishist_Mothers_Day_2017_mean,mer+hishist_Mothers_Day_2018_mean,mer+hishist_Valentine_Day_2017_mean,mer+hishist_amount_month_ratio_max,mer+hishist_amount_month_ratio_mean,mer+hishist_amount_month_ratio_min,mer+hishist_amount_month_ratio_skew,...,mer+hishist_purchase_date_diff,mer+hishist_purchase_date_uptomin,mer+hishist_purchase_date_uptonow,mer+hishist_weekday_mean,mer+hishist_weekend_mean,mer+hishist_weekofyear_max,mer+hishist_weekofyear_mean,mer+hishist_weekofyear_min,mer+hishist_weekofyear_nunique,mer+hishist_subsector_id_nunique
merchant_id,,,,,,,,,,,,,,,,,,,,,
M_ID_000ab1982d,0.000000,0.000000,0.000000,0.000000,94.000000,0.000000,3.846154,3.846154,3.846154,0.000000,...,0.0,389.0,389.0,1.000000,2.000000,0.000000,6.000000,6.0,6.0,1.0
M_ID_0088e0fbb9,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,13.333333,13.333333,13.333333,0.000000,...,0.0,438.0,438.0,1.000000,2.000000,0.000000,51.000000,51.0,51.0,1.0
M_ID_009074f3ff,0.000000,0.000000,0.000000,0.000000,66.000000,0.000000,1.485000,1.485000,1.485000,0.000000,...,0.0,360.0,360.0,1.000000,2.000000,0.000000,10.000000,10.0,10.0,1.0
M_ID_00a6ca8a8a,9.285714,8.389610,7.572872,1.558442,74.344875,2.077922,97.553333,15.315403,1.322624,9.126388,...,564.0,1245.0,680.0,11.454545,3.478355,52.277778,71.181818,18.5,35.0,16.0
M_ID_00b018b01e,0.000000,0.000000,0.000000,0.000000,24.000000,0.000000,1.658333,1.658333,1.658333,0.000000,...,0.0,319.0,319.0,1.000000,2.000000,0.000000,16.000000,16.0,16.0,1.0
M_ID_00c43dc392,7.000000,0.000000,38.000000,0.000000,0.000000,0.000000,1.312500,1.312500,1.312500,0.000000,...,0.0,471.0,471.0,1.000000,4.000000,0.000000,46.000000,46.0,46.0,1.0
M_ID_0210abecdc,0.000000,67.000000,0.000000,0.000000,0.000000,0.000000,33.081429,33.081429,33.081429,0.000000,...,0.0,574.0,574.0,1.000000,5.000000,1.000000,31.000000,31.0,31.0,1.0
M_ID_025bb0e805,0.000000,65.000000,0.000000,0.000000,0.000000,0.000000,1.373333,1.373333,1.373333,0.000000,...,0.0,572.0,572.0,1.000000,0.000000,0.000000,32.000000,32.0,32.0,1.0
M_ID_02c50d7b83,0.000000,0.000000,0.000000,0.000000,63.000000,0.000000,13.333333,13.333333,13.333333,0.000000,...,0.0,358.0,358.0,1.000000,5.000000,1.000000,10.000000,10.0,10.0,1.0
